In [83]:
import pymysql
import pandas as pd
import random


In [84]:
db = pymysql.connect(user='root', password='1234', host='127.0.0.1', db='Movie_DB', cursorclass=pymysql.cursors.DictCursor)

cursor = db.cursor()

query = "select * from movie_db.영화"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_movie_df=pd.DataFrame(db_export)
db_movie_df

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,내 여동생과 섹스해 줘,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
1,불륜에 중독된 변태 유부녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
2,남동생마저 노리는 욕정녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
3,성욕이 너무 강한 글래머,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
4,처음 본 대물에 젖어버린 유부녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
...,...,...,...,...,...,...,...,...,...
65523,최종면접,Last Interview,2012.0,한국,단편,드라마,기타,현문섭,None
65524,플라멩코 소녀,Flamenco Nina,2013.0,한국,단편,드라마,기타,이찬호,None
65525,KTX 특별열차,KTX Extra,2009.0,한국,단편,드라마,기타,정수진,None
65526,무더운 하루,None,2013.0,한국,단편,기타,기타,None,None


In [85]:
query = "select * from movie_db.감독"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_director_df=pd.DataFrame(db_export)
db_director_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,20396007,이세영,None,감독,나는 아빠다|대한민국 1%
1,20331280,이용남,None,감독,행복의 발견|유 돈 노우|장마|느루
2,20395966,기시 요시유키,Yoshiyuki Kishi,감독,정욕
3,20395965,니콜라우스 게이할터,Nikolaus Geyrhalter,감독,아름다운 쓰레기 여행
4,20395964,키무라 별 르무안,KIMURA byol LEMOINE,감독,입양아에서 입양인으로: 한국 국제 입양의 70년
...,...,...,...,...,...
24927,10090862,휴버트 프랭크,None,감독,카프리의 깊은밤 3
24928,10090871,흐라픈 군라프슨,Hrafn Gunnlaugsson,감독,사랑이 눈뜰 때
24929,10090882,히가시 요이치,Yoichi Higashi,감독,"러브 호텔 무삭제판|술이 깨면 집에 가자|간호사, 나츠코의 뜨거운 여름|나의 조교일..."
24930,10090896,히구친스키,Higuchinsky,감독,소용돌이


In [86]:
query = "select * from movie_db.배우"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_actor_df=pd.DataFrame(db_export)
db_actor_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,20395989,김정연,None,배우,착한 아이 사탕이|제6회 김포국제청소년영화제\t 폐막작
1,20392256,김수진,None,배우,달짝지근해: 7510|브릿지
2,20395987,아디나 바잔,Adina BAZHAN,배우,다우렌의 결혼
3,20331052,이은미,LEE Eun-mi,배우,거짓말의 색은 노랑|블라인드|보속|내 동생|시선|타래 |목격담|김다육
4,20395962,정다민,JUNG Da-min,배우,수학영재 형주
...,...,...,...,...,...
47668,10090907,히다카 노리코,Noriko Hidaka,배우,극장판 이누야샤 4 - 홍련의 봉래도|이웃집 토토로
47669,10090917,히데오 모리,None,배우,요코하마 메리
47670,10090980,히로시 시미즈,Hiroshi Shimizu,배우,포스트맨 블루스
47671,10091039,히토토 요,Yo Hitoto,배우,카페 뤼미에르


In [87]:
# 고유한 장르 목록을 확인하는 코드
def check_genre():
    unique_genres = db_movie_df['장르'].unique()
    genreList = []
    for genre in unique_genres:
        try:
            x = genre.split(',')
            genreList.extend(x)
        except AttributeError:
            pass

    genreList=list(set(genreList)) # 장르의 종류
    genreList.remove('기타')
    return genreList

In [88]:
# 장르별 영화추천
def recommend_genre(genre):

    # 장르가 '드라마'인 영화 선택
    movies = db_movie_df[db_movie_df['장르'] == genre]['영화명'].tolist()

    # 장르가 없는 영화인 경우
    if not movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = random.choice(movies)
        return db_movie_df[db_movie_df['영화명'] == recommended_movie]

In [89]:
# 고유한 제작연도를 확인하는 코드
def check_years():
    unique_years = db_movie_df['제작연도'].unique()
    unique_years=list(set(unique_years))
    return unique_years

In [90]:
#제작연도별 영화 추천
def recommend_year(year):
    movies = db_movie_df[db_movie_df['제작연도'] == year]['영화명'].tolist()

    if not movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = random.choice(movies)

        return db_movie_df[db_movie_df['영화명'] == recommended_movie]

In [91]:
# 배우-filmoNames에 데이터를 분리, 랜덤 출력
def separated_string(string):
    choices = string.split("|")
    randomMovieName = random.choice(choices)
    return randomMovieName

In [92]:
# 배우-배우가 한명일 때 실행될 함수
def onePeople(actor, df):

    actor_movies = df[df['peopleNm'] == actor]['filmoNames'].tolist()
    # 검색 배우의 출연작이 없으면 None 리턴
    if not actor_movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = separated_string(random.choice(actor_movies))
        return recommended_movie

In [93]:
# 배우-한명이 아닐 때 실행될 함수
def twoPeople(user_input_actor, df, movie_name):
    
    # 새로운 조건으로 데이터 검색
    query = "SELECT * FROM 배우 WHERE peopleNm = %s AND filmoNames LIKE %s"
    cursor.execute(query, (user_input_actor, f'%{movie_name}%'))
    twoPeopleActorsData = cursor.fetchall()

    # 새로운 데이터프레임 생성
    df = pd.DataFrame(twoPeopleActorsData)
    actor_movies = df[df['peopleNm'] == user_input_actor]['filmoNames'].tolist()
    
    if not actor_movies:
        return None
    else:
        recommended_movie = separated_string(random.choice(actor_movies))
        return recommended_movie

In [94]:
# 배우-검색 결과 출력
def ActorPrint(user_input_actor):
    search_result = db_actor_df[db_actor_df['peopleNm'] == user_input_actor]

    if not search_result.empty:

        # 입력받은 배우가 한명일 때
        if len(search_result) == 1:
            recommended_movie = onePeople(user_input_actor, db_actor_df)
        
            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_actor}인 배우가 출연한 영화가 없습니다.")

        # 입력받은 배우가 한명이 아닐 때 해당 배우 출연 영화명 입력받기
        else:
            movie_name = input("영화명을 입력하세요: ").strip()
            recommended_movie = twoPeople(user_input_actor, db_actor_df, movie_name)

            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_actor}인 배우가 출연한 영화가 없습니다.")
    else:
        print(f"입력한 이름 '{user_input_actor}'으로 검색된 배우가 없습니다.")

In [95]:

# 사용자 입력 받기
user_input_actor = ActorPrint(input("배우의 이름을 입력하세요: ").strip())

배우의 이름을 입력하세요:  김수현
영화명을 입력하세요:  수상한 그녀


추천 영화: 은밀하게 위대하게 (확장판)


In [96]:
# 감독-filmoNames에 데이터를 분리, 랜덤 출력
def separated_string(string):
    choices = string.split("|")
    randomMovieName = random.choice(choices)
    return randomMovieName

In [97]:
# 감독-감독이 한명일 때 실행될 함수
def onePeople(director, df):

    director_movies = df[df['peopleNm'] == director]['filmoNames'].tolist()
    # 검색 감독이 제작한 영화가 없으면 None 리턴
    if not director_movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = separated_string(random.choice(director_movies))
        return recommended_movie

In [98]:
# 감독-한명이 아닐 때 실행될 함수
def twoPeople(user_input_director, df, movie_name):
    
    # 새로운 조건으로 데이터 검색
    query = "SELECT * FROM 감독 WHERE peopleNm = %s AND filmoNames LIKE %s"
    cursor.execute(query, (user_input_director, f'%{movie_name}%'))
    twoPeopleDirectorData = cursor.fetchall()

    # 새로운 데이터프레임 생성
    df = pd.DataFrame(twoPeopleDirectorData)
    director_movies = df[df['peopleNm'] == user_input_director]['filmoNames'].tolist()
    
    if not director_movies:
        return None
    else:
        recommended_movie = separated_string(random.choice(director_movies))
        return recommended_movie

In [99]:
# 감독-검색 결과 출력
def DirectorPrint(user_input_director):
    search_result = db_director_df[db_director_df['peopleNm'] == user_input_director]

    if not search_result.empty:

        # 입력받은 감독이 한명일 때
        if len(search_result) == 1:
            recommended_movie = onePeople(user_input_director, db_director_df)
        
            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_director}인 감독이 제작한 영화가 없습니다.")

        # 입력받은 감독이 한명이 아닐 때 해당 감독 제작 영화명 입력받기
        else:
            movie_name = input("영화명을 입력하세요: ").strip()
            recommended_movie = twoPeople(user_input_director, db_director_df, movie_name)

            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_director}인 감독이 제작한 영화가 없습니다.")
    else:
        print(f"'{user_input_director}'으로 검색된 감독이 없습니다.")

In [100]:

# 사용자 입력 받기
user_input_director = DirectorPrint(input("감독의 이름을 입력하세요: ").strip())

감독의 이름을 입력하세요:  봉준호


추천 영화: 괴물


In [116]:
user=0
movie_store_df = pd.DataFrame({})
while True:
    start_m=''' 안녕하세요!! 시네마 미슐랭가이드입니다.^^ 어떤 서비스를 원하시나요?\n
    1. 영화 장르별 추천
    2. 영화 개봉연도별 추천
    3. 감독별 영화 추천
    4. 배우별 영화 추천
    5. 리뷰(수정해주세요)
    6. 찜목록 보기
    0. 프로그램 종료하기'''
    print(start_m)
    user=int(input())
    if user==0:
        print('프로그램을 종료합니다.')
        break
    elif user==1:
        print(check_genre())
        user_genre=input('장르별 영화 추천을 해드립니다. 어떤 장르를 원하시나요?\n\n')
        res=recommend_genre(user_genre)
        if not res.empty:
            print('*****이 영화를 추천해드릴게요!!*****')
            print(res)
            print()
            user_genre2=input('이 영화를 찜하시겠습니까? 1.Yes 0.No 종료\n\n')
            if user_genre2==1:
                movie_store_df = pd.concat([movie_store_df, res])
            else:
                pass
        else:
            print('없는 장르입니다.')
        
        
    elif user==2:
        print(check_years())
        user_year=int(input('개봉연도별 영화 추천을 해드립니다. 어떤 연도를 원하시나요?\n\n'))
        res=recommend_year(user_year)
        if not res.empty:
            print('*****이 영화를 추천해드릴게요!!*****')
            print(res)
            print()
            user_year2=input('이 영화를 찜하시겠습니까? 1.Yes 0.No 종료\n\n')
            if user_year2==1:
                movie_store_df = pd.concat([movie_store_df, res])
            else:
                pass
        else:
            print('없는 개봉연도입니다.')

    elif user==3:
        pass
        
    elif user==4:
        pass
    elif user==5:
        pass
        
    elif user==6:
        print(movie_store_df)
        user_store=int(input('저장하신 찜목록입니다. 삭제할 데이터가 있습니까? 1. Yes 0. No 종료\n\n'))
        if user_store==1:
            user_movie=input('삭제할 영화제목을 입력해주세요.\n\n')
            movie_store_df = movie_store_df[movie_store_df['영화제목'] != user_movie]
            print(movie_store_df)
            print('삭제되었습니다.\n\n')
        else:
            pass

 안녕하세요!! 시네마 미슐랭가이드입니다.^^ 어떤 서비스를 원하시나요?

    1. 영화 장르별 추천
    2. 영화 개봉연도별 추천
    3. 감독별 영화 추천
    4. 배우별 영화 추천
    5. 리뷰(수정해주세요)
    6. 찜목록 보기
    0. 프로그램 종료하기


 2


[nan, 1874.0, 1895.0, 1897.0, 1900.0, 1902.0, 1903.0, 1904.0, 1905.0, 1906.0, 1907.0, 1912.0, 1914.0, 1915.0, 1916.0, 1918.0, 1919.0, 1920.0, 1921.0, 1922.0, 1923.0, 1924.0, 1925.0, 1926.0, 1927.0, 1928.0, 1929.0, 1930.0, 1931.0, 1932.0, 1933.0, 1934.0, 1935.0, 1936.0, 1937.0, 1938.0, 1939.0, 1940.0, 1941.0, 1942.0, 1943.0, 1944.0, 1945.0, 1946.0, 1947.0, 1948.0, 1949.0, 1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2023.0, 2024.0, 2025.0]


개봉연도별 영화 추천을 해드립니다. 어떤 연도를 원하시나요?

 2022


*****이 영화를 추천해드릴게요!!*****
         영화명               영화명(영문)    제작연도 제작국가  유형     장르 제작상태   감독   제작사
13900  2차 송환  The 2nd Repatriation  2022.0   한국  장편  다큐멘터리   개봉  김동원  푸른영상



...

Empty DataFrame
Columns: []
Index: []


저장하신 찜목록입니다. 삭제할 데이터가 있습니까? 1. Yes 0. No 종료

 0


 안녕하세요!! 시네마 미슐랭가이드입니다.^^ 어떤 서비스를 원하시나요?

    1. 영화 장르별 추천
    2. 영화 개봉연도별 추천
    3. 감독별 영화 추천
    4. 배우별 영화 추천
    5. 리뷰(수정해주세요)
    6. 찜목록 보기
    0. 프로그램 종료하기


 0


프로그램을 종료합니다.


In [117]:
movie_store_df = pd.DataFrame({})

In [127]:
res=recommend_year(2022)

In [128]:
movie_store_df = pd.concat([movie_store_df, res])

In [129]:
print(movie_store_df)

                        영화명       영화명(영문)    제작연도 제작국가    유형      장르  제작상태  \
4036           무협대호: 학살의 칼날  Mutant Tiger  2022.0   중국    장편      액션    기타   
14461                 동물의 눈   Olho Animal  2022.0  프랑스    장편   다큐멘터리    기타   
12200       세상 참 예쁜 오드리(가제)        Audrey  2022.0   한국    장편  가족,드라마  후반작업   
18548  제11회 광주독립영화제: 단편 초청3          None  2022.0   한국  옴니버스      기타    기타   

         감독           제작사  
4036   None  (주)스튜디오디에이치엘  
14461  None          None  
12200   이영국          None  
18548  None          None  
